# Redis Basics Tutorial

Introduction aux bases de Redis avec notre dataset de films.

In [ ]:
from utils.redis_client import RedisClient

# Initialiser la connexion
client = RedisClient()

# Tester la connexion
print("Connection test:", client.ping())

# Récupérer un film
movie = client.get_movie(1)
print("\nMovie details:", movie)